# Wasserstein GAN - Improved

ref.:
GULRAJANI, Ishaan et al.  
Improved training of wasserstein gans.  
In: Advances in neural information processing systems. 2017. p. 5767-5777.

![new objective](wgan_gp_objective.png)